In [2]:
pip install scikit-surprise
pip install jsonlines

     ---------------------------------------- 0.0/772.0 kB ? eta -:--:--
     - -------------------------------------- 30.7/772.0 kB ? eta -:--:--
     ----- -------------------------------- 112.6/772.0 kB 1.7 MB/s eta 0:00:01
     ------------ ------------------------- 245.8/772.0 kB 1.9 MB/s eta 0:00:01
     ----------------- -------------------- 348.2/772.0 kB 2.0 MB/s eta 0:00:01
     ----------------- -------------------- 348.2/772.0 kB 2.0 MB/s eta 0:00:01
     ------------------- ------------------ 399.4/772.0 kB 1.5 MB/s eta 0:00:01
     ---------------------- --------------- 450.6/772.0 kB 1.5 MB/s eta 0:00:01
     ---------------------- --------------- 450.6/772.0 kB 1.5 MB/s eta 0:00:01
     ---------------------- --------------- 450.6/772.0 kB 1.5 MB/s eta 0:00:01
     ----------------------------------- -- 716.8/772.0 kB 1.6 MB/s eta 0:00:01
     -------------------------------------- 772.0/772.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing 

In [15]:
import pandas as pd
import json
import gzip
import time
# Load the data
df = pd.read_csv('df_ca.csv')
df = df.drop('Unnamed: 0', axis=1)
df.head(20)

,rating,reviewerName,reviewText,categories,gPlusPlaceId,unixReviewTime,reviewTime,gPlusUserId,gps,price,dtime,hour,minute,second
0,4.0,william spindler,Best War Wanton soup in Red Bluff,"['Asian Restaurant', 'Chinese Restaurant']",106591714648856494903,1394669496,"Mar 12, 2014",100000032416892623125,"[40.179159, -122.236162]",$$$,2014-03-13 00:11:36,0,11,36
1,5.0,william spindler,This is a review that is long overdo. I've bee...,"['European Restaurant', 'Italian Restaurant', ...",109420033090810328045,1394826388,"Mar 14, 2014",100000032416892623125,"[40.178074, -122.235234]",$$,2014-03-14 19:46:28,19,46,28
2,5.0,william spindler,"Long time favorite Mexican food, always consis...",['Mexican Restaurant'],115827996910815192564,1394669713,"Mar 12, 2014",100000032416892623125,"[40.175064, -122.242574]",$$,2014-03-13 00:15:13,0,15,13
3,5.0,Anne Mason,Great coffee and location! Check out the used ...,"['Espresso Bar', 'Coffee Shop']",108196901293702895067,1379187049,"Sep 14, 2013",100000106576186066497,"[38.030536, -121.884024]",$$$,2013-09-14 19:30:49,19,30,49
4,5.0,Anne Mason,"I've been here dozens of times, and I'll keep ...","['Sushi Restaurant', 'Asian Restaurant', 'Japa...",111971088396054239194,1394848761,"Mar 14, 2014",100000106576186066497,"[37.961078, -121.748886]",$$,2014-03-15 01:59:21,1,59,21
5,3.0,William Corcuera,Finally came back to this place after so many ...,"['Latin American Restaurant', 'Mexican Restaur...",101788547508969264434,1347079555,"Sep 7, 2012",100000122158721897485,"[33.20043, -117.331493]",$$$,2012-09-08 04:45:55,4,45,55
6,5.0,William Corcuera,I live in sandiego its about an hour drive and...,['Chicken Restaurant'],101959623187827102951,1388002613,"Dec 25, 2013",100000122158721897485,"[33.680515, -117.90633]",$$$,2013-12-25 20:16:53,20,16,53
7,2.0,William Corcuera,I ordered 4 rolled tacos. when I received them...,"['Mexican Restaurant', 'Latin American Restaur...",105242172534828461225,1324327546,"Dec 19, 2011",100000122158721897485,"[33.131197, -117.260452]",$$$,2011-12-19 20:45:46,20,45,46
8,3.0,adam porto,WELL I HAVE EATEN HERE SEVERAL TIMES AND EVERY...,"['American Restaurant', 'Dessert Restaurant']",101634997104301806688,1352400948,"Nov 8, 2012",100000174752447962893,"[34.112, -117.535357]",$$,2012-11-08 18:55:48,18,55,48
9,5.0,Jason Wagner,The service is good and they are nice. Everyth...,"['Vegetarian Restaurant', 'Eclectic Restaurant...",118165048892915833083,1308018773,"Jun 13, 2011",100000196778399872657,"[33.042048, -117.29286]",$$,2011-06-14 02:32:53,2,32,53


In [16]:
file_path = 'data/places.clean.json.gz'
def read_reviews(file_path):
    with gzip.open(file_path) as file:
        for line_number, line in enumerate(file, start=100000):
            # review = json.loads(eval(line))
            review = eval(line)
            yield review

In [17]:
def find_PlaceId(name):
    file_path = 'data/places.clean.json.gz'
    gplusPlaceId=""
    for review in read_reviews(file_path):
        gplusPlaceName=""
        message_value = json.dumps(review)
        if review["name"] == name:
            gplusPlaceId = review["gPlusPlaceId"]
            break
    return gplusPlaceId

In [28]:
import pickle

def find_name(place_id):
    input_file_path = 'data/place_name_dict/place_name_dict.pkl'
    # Đọc place_name_dict từ tệp
    with open(input_file_path, 'rb') as file:
        place_name_dict = pickle.load(file)
    place_name = place_name_dict.get(place_id)
    return place_name

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
def find_similar_categories(input_category, categories, ratings, reviewer_names, gps):
    # Combine the input category, categories, ratings, and reviewer names into a list
    all_categories = [(input_category, 1.0, "", "")] + list(zip(categories, ratings, reviewer_names, gps))
    
    # Create a TfidfVectorizer
    vectorizer = TfidfVectorizer()
    
    # Transform the categories into TF-IDF vectors
    tfidf_matrix = vectorizer.fit_transform([category for category, _, _, _ in all_categories])
    
    # Calculate the cosine similarity between the input category and all other categories
    similarity_scores = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1:])
    
    # Get the indices of the most similar categories
    most_similar_indices = similarity_scores.argsort()[0][::-1]
    
    # Sort the ratings and GPS based on the category indices
    sorted_ratings = sorted(zip(most_similar_indices, ratings, reviewer_names, gps), key=lambda x: x[1], reverse=True)
    
    unique_gps = set()
    unique_sorted_ratings = []
    
    # Filter out duplicate GPS values
    for index, rating, reviewer_name, gps_value in sorted_ratings:
        if gps_value not in unique_gps:
            unique_gps.add(gps_value)
            unique_sorted_ratings.append((index, rating, reviewer_name, gps_value))
    
    return most_similar_indices[:5], unique_sorted_ratings

### Place ID input 

In [29]:
import csv
import pandas as pd

# Read the data from a CSV file
data = pd.read_csv('df_ca.csv')

# Get the categories, ratings, reviewer names, and GPS columns
categories = data['categories'].tolist()
ratings = data['rating'].tolist()
reviewer_names = data['reviewerName'].tolist()
gps = data['gps'].tolist()
gPlusPlaceId = data['gPlusPlaceId'].tolist()

# Input gPlusPlaceId
input_gplus_place_id = input("Nhập gPlusPlaceId: ")

# Tìm danh mục (category) tương ứng với input_gplus_place_id
filtered_data = data[data['gPlusPlaceId'] == input_gplus_place_id]
index_input = filtered_data.index.tolist()[0]
input_category = filtered_data['categories'].iloc[0]
rating_input = filtered_data['rating'].iloc[0]
reviewername_input = filtered_data['reviewerName'].iloc[0]
Gps_input = filtered_data['gps'].iloc[0]
Place_name_input = find_name(input_gplus_place_id)

if input_category:
    # Find similar categories and their indices, along with sorted ratings
    similar_category_indices, sorted_ratings = find_similar_categories(input_category, categories, ratings, reviewer_names, gps)
    # Print and write the similar categories, indices, ratings, reviewer names, and GPS values in the desired format
    print("Indices, Similar Categories, Ratings, Reviewer Names, GPS, Place_ID and Name:")
    with open('result.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Index', 'Category', 'Rating', 'Reviewer Name', 'GPS', 'Place_ID', 'Place Name'])
        writer.writerow([index_input, input_category, rating_input, reviewername_input, Gps_input, input_gplus_place_id, Place_name_input])
        for index, rating, reviewer_name, gps_value in sorted_ratings[:5]:
            category = categories[index]
            gplus_place_id = gPlusPlaceId[index]
            place_name = find_name(gplus_place_id)
            writer.writerow([index, category, rating, reviewer_name, gps_value, gplus_place_id, place_name])
            print(f"{index:5}  {category:60} {rating:<5} {reviewer_name} {gps_value} {gplus_place_id} {place_name}")
else:
    print("Không tìm thấy gPlusPlaceId trong dữ liệu.")

Nhập gPlusPlaceId: 105242172534828461225
Indices, Similar Categories, Ratings, Reviewer Names, GPS, Place_ID and Name:
56368  ['Latin American Restaurant', 'Mexican Restaurant']          5.0   william spindler [40.178074, -122.235234] 115374409149458089265 Salsa Tacos Mexican Food
17551  ['Mexican Restaurant', 'Latin American Restaurant']          5.0   william spindler [40.175064, -122.242574] 104668746138130998884 Viva Zapatas Mexican Restaurant & Cantina
73609  ['Mexican Restaurant', 'Latin American Restaurant']          5.0   Anne Mason [38.030536, -121.884024] 105281764361123792485 Chevys
70917  ['Mexican Restaurant', 'Latin American Restaurant']          5.0   Anne Mason [37.961078, -121.748886] 104322525716556434210 Gordo Taqueria
22396  ['Mexican Restaurant', 'Latin American Restaurant']          5.0   William Corcuera [33.680515, -117.90633] 101131250308402320301 Alberto's Mexican Food
